In [ ]:
import warnings
warnings.filterwarnings('ignore')

import re
import os
import io
import sys
import json
import tqdm
import pickle
import urllib
import requests
import textwrap
import functools
import traceback
import itertools
import matplotlib
import contextlib
import numpy as np
import pandas as pd
from Bio import SeqIO
import seaborn as sns
import networkx as nx
import plotly.express as px
from functools import reduce
import matplotlib.pyplot as plt

from multiprocessing import Pool
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

from pygosemsim import graph
from pygosemsim import download
from pygosemsim import term_set
from pygosemsim import similarity

from utilities.funcs import *

# Variables

In [ ]:
goslim_subset = 'goslim_generic'
level = 'at2759'
odb_levels = 'https://v101.orthodb.org/download/odb10v1_levels.tab.gz'
levels = pd.read_table(odb_levels, compression='gzip', header=None)
levels[0] = levels[0].apply(lambda x: f'at{x}')
levels = dict(levels[[0,1]].values.tolist())
levelname = levels.get(level)
goaspects = ('P','C','F')

## File paths

In [ ]:
# INFILES

transitions_file = 'Eukaryota.transitions.mini'
clusters_raw_file = 'Eukaryota.clusters.mini'
ogunigo_file = f'external/og_uni_go_{levelname}_experimental_mini.tsv'
koog_file = 'external/ko_og_pathway.tsv'
# #OUTFILES

transitions_ogs_file = f'{transitions_file}.ogs'
clusters_file = f'{clusters_raw_file}.grouped'
allogs_file = f"{transitions_file.split('.')[0]}.ogs"
ogunigotrans_file = f'{transitions_file}.unigo.experimental'
pygosemsim_file = f'{ogunigotrans_file}.pygosemsim'

goslim_subset = 'goslim_generic.obo'
goobo_file = 'external/go.obo'
goslim_file = 'external/annotations.generic'

odbfile = 'external/odbinfos.pickle'

# LINKS

odb_og2genes = 'https://v101.orthodb.org/download/odb10v1_OG2genes.tab.gz'
odb_genexrefs = 'https://v101.orthodb.org/download/odb10v1_gene_xrefs.tab.gz'

# Processing files

In [ ]:
# transition pairs to single, most significant ogs
transitionstoogs(transitions_file).to_csv(transitions_ogs_file, sep='\t', index=None)

# processing raw clusters file --> dataframe
clustersfromraw(clusters_raw_file, transitions_file).to_csv(clusters_file, sep='\t', index=None)

# all ogs list
transitionsogs = transitionstoogs(transitions_file)
allogsdf = transitionsogs[['og','ogname']]
allogsdf.columns = ['og','name']

allogsdf.to_csv(allogs_file, sep='\t', index=None)

# GO

## GO Mapping

Decomment the following cell for the long version

In [ ]:
##### for the long version
# oguni_file = f'external/og_uni_{levelname}.tsv'
# ogunigo_file = f'external/og_uni_go_{levelname}.tsv'

# !wget external/http://release.geneontology.org/2020-01-01/annotations/goa_uniprot_all.gaf.gz
# goa = 'external/goa_uniprot_all.gaf.gz'

# xref = pd.read_table(odb_og2genes, compression='gzip', header=None)
# og2genes = pd.read_table(ogb_genexrefs, compression='gzip', header=None)
# og2genes = og2genes[og2genes[2]=='UniProt']
# merged = pd.merge(og2genes, xref.rename(columns={0:3,1:0}))
# merged2 = merged[merged[3].str.contains(level)]
# merged2.columns = ['gene','id','db','og']
# merged2.to_csv(oguni_file, sep='\t', index=None)

# !python3 utilities/oguni2chunk goa oguni_file ogunigo_file

# oggo = pd.read_table(ogunigo_file)
# ex = ['IDA', 'IMP', 'IPI', 'IEP', 'IGI']

# oggo = oggo[oggo['og'].isin(set(oggo[oggo['evidence'].isin(ex)]['og']))]

# oggo[~oggo['go'].isin(['GO:0008150', 'GO:0005575', 'GO:0003674'])
#     ].to_csv(ogunigo_file.replace('.tsv', '_experimental.tsv'), 
#              sep ='\t', index=False)

For the brief version (we'll attach these file in the folder 'external')

In [ ]:
trans = pd.read_table(transitions_file)
clu = pd.read_table(clusters_file)
ogs = set(trans['Orthogroup1'].tolist()+trans['Orthogroup2'].tolist())

# oguni = pd.read_table('../../og_uni_Eukaryota.tsv')
# oguni[oguni['og'].isin(set(trans['Orthogroup1'].tolist()+trans['Orthogroup2'].tolist()))
#      ].to_csv('external/og_uni_Eukaryota_mini.tsv', sep='\t', index=None)

# ogunigo = pd.read_table('../../og_uni_go_Eukaryota_experimental_filt.tsv')
# ogunigo[ogunigo['og'].isin(ogs)
#        ].to_csv(ogunigo_file, sep='\t', index=None)

transitions = pd.read_table(transitions_file)
oggo = pd.read_table(ogunigo_file)
oggo = oggo[oggo['og'].isin(set(transitions['Orthogroup1'].tolist()+
                                transitions['Orthogroup2'].tolist()))]
oggo.to_csv(ogunigotrans_file, sep='\t', index=None)

## PyGOSemSim

In [ ]:
try:
    download.obo("go-basic")
    download.download("goslim_chembl.obo",
        "http://www.geneontology.org/ontology/subsets/goslim_chembl.obo")
    G = graph.from_resource("go-basic")
    G_chembl = graph.from_resource("goslim_chembl")
    similarity.precalc_lower_bounds(G)
except ValueError:
    pass

def calcsim(indexgo1go2):
    try:
        sf = functools.partial(term_set.sim_func, 
                               G, similarity.resnik)
        si = term_set.sim_bma(indexgo1go2[1], 
                              indexgo1go2[2], sf)
    except:
        si = 0
    return [indexgo1go2[0], si]

transitions = pd.read_table(transitions_file)
oggo_df = pd.read_table(ogunigotrans_file)
oggo = dict(oggo_df.groupby('og')['go'].apply(
    lambda x: list(set(x))).reset_index().values)
oggo_bp = dict(oggo_df[oggo_df['type']=='P'].groupby('og')[
    'go'].apply(lambda x: list(set(x))).reset_index().values)
oggo_cc = dict(oggo_df[oggo_df['type']=='C'].groupby('og')[
    'go'].apply(lambda x: list(set(x))).reset_index().values)
oggo_mf = dict(oggo_df[oggo_df['type']=='F'].groupby('og')[
    'go'].apply(lambda x: list(set(x))).reset_index().values)
dfs = []
for OGGO, typ in zip([oggo_bp, oggo_cc, oggo_mf],
                     ['BP', 'CC', 'MF']):
    oggo_df = transitions[['Orthogroup1','Orthogroup2']].applymap(
        lambda x: OGGO.get(x)).reset_index()
    oggo_df.columns = ['index','go1', 'go2']
    oggo_df = oggo_df.dropna()
    simscore = Pool(500).map(calcsim, oggo_df.values.tolist())
    simscore = dict(simscore)
    oggo_df['simscore'] = oggo_df['index'].apply(
        lambda x: simscore.get(x))
    oggo_df = oggo_df.drop('index', axis=1)
    oggo_df = oggo_df.reset_index(drop=True).rename(
        columns={'simscore':f'{typ.lower()}sim', 
                 'go1':f'go1_{typ.lower()}', 
                 'go2':f'go2_{typ.lower()}'})
    oggo_df[f'len_go1_{typ.lower()}'] = oggo_df[
        f'go1_{typ.lower()}'].apply(len)
    oggo_df[f'len_go2_{typ.lower()}'] = oggo_df[
        f'go2_{typ.lower()}'].apply(len)
    dfs.append(pd.concat([transitions.loc[
        list(simscore.keys())].reset_index(drop=True), 
                          oggo_df], axis=1))
total = (dfs[0]
 .merge(dfs[1], how='outer')
 .merge(dfs[2], how='outer')
)
total.to_csv(pygosemsim_file, sep='\t', index=False)

### Figure 3a

In [ ]:
save = False
title = 'GO experimental similarity vs p-value'

df = pd.read_table(pygosemsim_file)

ths = list(reversed(list(map(lambda x: 10**(-x/100), 
                             range(0,2000,100)))[3:]))[:-1]
chunks, ts = [],[]
for n, t in   enumerate(ths):
    if n == 0:
        chunk = df[df['p']<=t]
    if n == len(ths)-1:
        chunk = df[df['p']>t]
    else:
        try:
            chunk = df[df['p'].between(t, ths[n+1], 
                                       inclusive='left')]
        except:
            pass
    chunks.append(chunk)
    ts.append("{:.0e}".format(t))
    
columns = df.columns[df.columns.str.contains('sim')]
lens = list(map(lambda x: len(x), chunks))*len(columns)
scatter, line = [], []
for column in columns:
    name = column.rstrip('sim').upper()
    c = 100
    df1 = df[(df[f"len_go1_{column}".replace('sim', '')]>n)&
             (df[f"len_go2_{column}".replace('sim', '')]>n)]
    ser = pd.Series([len(d[d[column]>=df1[column].mean()])/len(d) 
                     if len(d) != 0 else 0 
                     for d in chunks]).reset_index()
    ser['type'] = name
    ser['slices'] = ts
    scatter.append(ser)
    
lab = 'Fraction > mean score'
data = pd.concat(scatter).reset_index(drop=True)
data = data.rename(columns={0: lab})
data['slices'] = data['slices'].astype(str)
data['lengths'] = lens
data.columns = ['Index','Fraction > mean score', 
                'GO class', 'Slice', 'Numerosity']
data['Log(N)'] = data['Numerosity'].apply(lambda x: np.log10(x))

In [ ]:
import math
import operator

factor = 3.1/4

plt.rcParams['axes.linewidth'] = 0.8
plt.rcParams['lines.linewidth'] = 0.8
plt.rcParams['font.size'] = 8
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['legend.fontsize'] = 6

fig, ax = plt.subplots(figsize=(3.1, 3.1), dpi=100) 

for typ, col, zo in zip(['CC','BP','MF'],[(0.75,0.75,0),
                                          (0,0.75,0.75),
                                          (0.75,0,0.75)],
                        [1,2,3]):

    plt.plot(data[data['GO class']==typ]['Slice'], 
             data[data['GO class']==typ][lab], 
             label=typ, color=col, zorder=zo)
    plt.scatter(data[data['GO class']==typ]['Slice'], 
                data[data['GO class']==typ][lab], 
                label=typ, color=col, linewidths=0,
                sizes=[i*factor*i*factor*math.pi 
                       for i in data[data['GO class']==typ
                                    ]['Log(N)']], zorder=zo)


scatter = plt.scatter([0,1,2], [0,1,2], color=(1,1,1),
                  sizes=[x*factor*x*factor*math.pi 
                         for x in (2,3,4)], zorder=0)

handles, labels = ax.get_legend_handles_labels()
hl = sorted(zip(handles, labels), key=operator.itemgetter(1))
hl = [hl[0],hl[2],hl[4]]
handles2, labels2 = zip(*hl)
legend3 = ax.legend(handles2, labels2, loc=(0.7,0.75), 
                    title='GO aspect', frameon=False)

handles, labels = scatter.legend_elements(
    prop="sizes", alpha=0.2, markeredgewidth=0)
labels = [2,3,4]
legend2 = ax.legend(handles, labels, loc=(0.73,0.5), 
                    title="Log(N)", frameon=False)
plt.gca().add_artist(legend3)

plt.xticks(rotation=-45, ha="left", rotation_mode="anchor")
ax.spines[['top', 'right']].set_visible(False)

plt.xlabel("p-value (slice)")
plt.ylabel("$N_{score>mean}$ / N")

ax.set_xticklabels(labels=["$10^{-tag}$".replace(
    'tag', str(s).split('-')[1]) for s in 
                           data[data['GO class']==typ]['Slice']])

plt.ylim(0,1)

save = True
if save == True:
    fig.tight_layout()
    fig = ax.get_figure()
    fig.savefig(f'images/{pygosemsim_file}.pdf')

## GO SLIM

In [ ]:
!wget http://purl.obolibrary.org/obo/go.obo -P external/

In [ ]:
!wget https://github.com/owlcollab/owltools/releases/download/2020-04-06/owltools -P utilities/
!chmod +x utilities/owltools

In [ ]:
# retrieve all GOs available from go.obo file
mini, minimini = [], []
for line in open(goobo_file).readlines()[29:]:
    if line.startswith('id: GO'):
        mini.append(minimini)
        minimini = []
    minimini.append(line)
gos = pd.DataFrame([[m[0].strip().split('id: ')[1],
                     m[1].strip().split('name: ')[1],
                     m[2].strip().split('namespace: ')[1]] 
                    for m in mini[1:]]).drop_duplicates(0)
gos = gos[~gos[1].str.contains('obsolete')]
gos = gos.reset_index(drop=True).reset_index()
gos.columns = ['index','go','goname','gotype']
gos['gotype'] = gos['gotype'].apply(lambda x: 
                                    'P' if 'biological' in x else 
                                    'F' if 'molecular' in x else 
                                    'C' if 'component' in x else None)
gonames = dict(gos[['go','goname']].values)
gotypes = dict(gos[['go','gotype']].values)

# GO to GO slim using OWLTOOLS (https://github.com/owlcollab/owltools)
def gostoslim(goobo, outfile, subset, gos):

    print('!gaf-version: 2.0', file=open(outfile, 'w'))
    for row in gos.values.tolist():
        a = list(map(str, [' ']*16))
        a[1], a[4] = str(row[0]), row[1]
        print('\t'.join(a), file=open(outfile, 'a'))
    
    with contextlib.redirect_stdout(io.StringIO()):
        !utilities/owltools {goobo} --gaf {outfile} --map2slim --subset {subset} --write-gaf {outfile}.gaf
    
    slim = pd.read_table(outfile+'.gaf', comment='!', header=None)
    slim = slim[[1,4]]
    slim.columns = ['index','go']
    slim['index'] = slim['index'].apply(lambda x: int(x[1:]))
    
    return slim

slim = gostoslim(goobo_file, goslim_file, 
                 os.path.basename(goslim_subset.replace('.obo', '')
                            ), gos).assign(db=goslim_subset)

# re-mapping any missing goslim_generic GO to goslim_PIR (more specific)
if 'generic' in goslim_subset:    
    missing = pd.merge(gos.rename(columns={0:'go'}), 
                       slim.rename(columns={'go':'goslim'}),how='outer')
    missing = missing[missing['goslim'].isna()][['index','go']]
    
    slim_pir = gostoslim(goobo_file, goslim_file.replace('.generic', '.pir'), 
                      os.path.basename(goslim_subset.replace(
                          '_generic.obo', '_pir')), missing
                     ).assign(db=goslim_subset.replace(
        '_generic.obo', '_pir'))
    final = pd.merge(gos, pd.concat(
        [slim, slim_pir]).rename(
        columns={'go':'goslim'}),on='index',how='outer').dropna()
    final['goslimname'] = final['goslim'].apply(lambda x: gonames.get(x))
    final['goslimtype'] = final['goslim'].apply(lambda x: gotypes.get(x))
    slim = final

slim = slim[['go','gotype','goname','goslim','goslimtype','goslimname','db']]
slim.to_csv(f'{goslim_file}.tsv', index=None, sep='\t')

### Orthogroups GO slim mapping

In [ ]:
ogunigo = pd.read_table(ogunigo_file)
slim = pd.read_table(f'{goslim_file}.tsv')

ogunigoslim = pd.merge(ogunigo, slim, how='outer'
                      ).dropna().drop_duplicates()
ogunigoslim.to_csv(f'{ogunigotrans_file}.slim', 
                   sep='\t', index=None)

transitionsogs = transitionstoogs(transitions_file)
goslim = pd.read_table(f'{ogunigotrans_file}.slim')

singlego = pd.merge(transitionsogs, goslim, on='og')
singlego = singlego.drop_duplicates([
    'og','goname','goslimname'])

# exclude macro GO classes
singlego = singlego[~singlego['goslimname'].isin(
    ['biological_process','cellular_component',
     'molecular_function'])]

# transitions pvalue to -log10 pvalue
singlego['log10pval'] = singlego['pval'].apply(
    lambda x: -np.log10(x))
singlego['log10adjpval'] = singlego['adjpval'].apply(
    lambda x: -np.log10(x))
final = singlego.reset_index(drop=True).sort_values(
    'log10pval', ascending=False)

# using scalepval function to force pvalues in a 
# user defined range (-6:-2, 2:6; >6 == 6)
final2 = scalepval(final, 'log10pval', 2, None)

final2.to_csv(str(f'{ogunigotrans_file}.slim'
                 ).replace('transitions','ogs'), 
              sep='\t', index=None)

#-log10 pvalues distribution after scaling them
final2['log10pvalscaled'].plot.hist(
    bins=100, figsize=(3.1, 3.1))

### Orthogroup ranking files

In [ ]:
# write a pval based ranking file containing orthogroups and their GO annotations
for goclass in goaspects:
    finalfilt = final2[final2['goslimtype']==goclass]
    finalfilt = finalfilt[['og','log10pvalscaled',
                           'goslim','goslimname',
                           'ogname']]
    finalfilt.columns = ['name','score','feature',
                         'description','extraname']
    finalfilt.drop_duplicates(['name','feature',
                               'description']
        ).to_csv(f"{f'{ogunigotrans_file}.slim'.replace('transitions','ogs')}.{goclass}.ranking", 
                 sep='\t', index=None)

### GO Enrichment

In [ ]:
# enrichemnt analysis using clusterProfiler library in R (https://github.com/YuLab-SMU/clusterProfiler)
for goclass in goaspects:
    file = f"{f'{ogunigotrans_file}.slim'.replace('transitions','ogs')}.{goclass}.ranking"
    !utilities/enrich.R --input_file {file} --pval_cutoff 1

### Figure 3b

In [ ]:
colorsfrommap = [plt.cm.jet(n)[0:3] for n in np.array(list(range(0,12)))/11]

newcolors = np.array([(0.75,0.75,0,i/255) for i in range(255)])
newcmp1 = matplotlib.colors.ListedColormap(newcolors)
newcolors = np.array([(0,0.75,0.75,i/255) for i in range(255)])
newcmp2 = matplotlib.colors.ListedColormap(newcolors)
newcolors = np.array([(0.75,0,0.75,i/575) for i in range(255)])
newcmp3 = matplotlib.colors.ListedColormap(newcolors)

In [ ]:
pval = 0.05
maxgr = 30
save = True

plt.rcParams['axes.linewidth'] = 0.8
plt.rcParams['lines.linewidth'] = 0.8
plt.rcParams['font.size'] = 8
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['legend.fontsize'] = 6

def plotenrichment(enrfile, rankfile, colormap, title, 
                   pvalthreshold, genesizelimit, save=False):
    
    def customwrap(s, width=20):
        return "\n".join(textwrap.wrap(s,width=width))

    df = pd.read_table(enrfile, sep=',').drop(
        'Unnamed: 0', axis=1).dropna()

    df = df[df['pvalue']<=pvalthreshold]
    df = df.sort_values('NES', ascending=False)
    df = df.reset_index(drop=True)
    
    df['Description'] = df['Description'].apply(
        lambda x: 'nucleobase-cont. small mol. met. proc.' if 'nucleobase-containing small molecule metabolic process' in x 
        else 'gen. of precursor metabol. and energy' if 'generation of precursor metabolites and energy' in x
        else x)

    df['Description'] = df['Description'].apply(lambda x: customwrap(x))

    df['core_enrichment'] = df['core_enrichment'].apply(lambda x: x.split('/'))
    ognames = dict(pd.read_table(rankfile).drop_duplicates(
        'name')[['name','extraname']].values)
    df['core_enrichment_names'] = df['core_enrichment'].apply(
        lambda x: [ognames.get(i) for i in x])
    df['coreSize'] = df['core_enrichment'].apply(len)
    df['coreSizenorm'] = df['coreSize'].apply(
        lambda x: genesizelimit if x>=genesizelimit else x)

    fig, (cax,ax) = plt.subplots(ncols=2, figsize=(4,4), dpi=100, 
                                 gridspec_kw={'width_ratios':[0.05, 1]})

    x = df['Description'].values
    ##########
    # x = df['Description'].values.tolist()+[' ','  ','   ']
    ##########
    x = np.array(['nucleob.-containing small mol. metabolic process' 
                  if 'nucleobase-containing small molecule metabolic process' 
                  in i else i for i in x])
    y = df['NES'].values
    ##########
    # y = df['NES'].values.tolist()+[0,0,0]
    ##########
    c = df['coreSizenorm'].values

    colmap = colormap
    colors1 = colmap(c/c.max())
    sm = plt.cm.ScalarMappable(cmap=colmap, 
                               norm=plt.Normalize(vmin=c.min(), vmax=c.max()))

    cbar = plt.colorbar(sm, cax)
    cbar.set_ticks(list(range(0,genesizelimit+1,int(genesizelimit/5))))
    cbar.set_ticklabels(list(range(
        0,genesizelimit,int(genesizelimit/5)))+[f'>{genesizelimit}'])
    cbar.outline.set_linewidth(0.8)
    cbar.set_label('core enrichment', loc='top', fontsize=6)

    ax.barh(x, y, color=colors1, edgecolor = colormap.colors[-1])
    ax.yaxis.tick_right()
    ax.tick_params(axis='both', which='major')
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines[['right','bottom']]
    ax.set_xlim(-4, 4)
    ax.set_xlabel('NES')

    cax.yaxis.tick_left()
    cax.tick_params(axis='both', which='major')

    plt.gca().invert_yaxis()
    plt.show()
    fig.tight_layout()

    if save == True:
        fig.savefig(f'images/{enrfile}.pdf')

for goclass in ['P','C','F']:
    rankfile = f"{f'{ogunigotrans_file}.slim'.replace('transitions','ogs')}.{goclass}.ranking"
    enrfile = f"{rankfile}.enriched"
    plotenrichment(enrfile, rankfile, newcmp1 if goclass == 'C' 
                   else newcmp2 if goclass == 'P' else newcmp3, 
                   'NES BP' if goclass == 'P' else 'NES CC' 
                   if goclass == 'C' else 'NES MF', pval, maxgr, save)

# KEGG

In [ ]:
# download per each orthogroup all available databases annotations from orthodb
!python3 utilities/oginfo.py -i {allogs_file} -o {odbfile} -p 4

In [ ]:
odbinfos = dict(pickle.load(open(odbfile, 'rb')))

# and store them in a dataframe
dfs = []
for o in odbinfos.keys():
    data = odbinfos.get(o)
    if len(data)>0:
        data = data['data']
        keys = [k for k in data.keys() if type(data.get(k)) == list]
        if len(keys)>0:
            dfs.append(pd.concat([pd.json_normalize(data.get(k)).assign(db=k) 
                            for k in keys]).assign(og=o))

dfs = pd.concat(dfs).drop(['name','type','count'], 
                          axis=1).reset_index(drop=True)

In [ ]:
clusters = pd.read_table(clusters_file)

# add orthodb annotations to each transitions
transitionsogs = transitionstoogs(transitions_file)
odbtransitions = pd.merge(dfs, transitionsogs, on='og')[['description','id','db','og','ogname']]

odbtransitions = pd.merge(odbtransitions, clusters[['og','cluster']], on='og', how='outer')
odbtransitions.to_csv(f'{transitions_file}.odb', sep='\t', index=None)

# same to each clusters
odbclusters = pd.merge(dfs, clusters[['cluster','n','og','name']])[['description','id','db','og','name','cluster','n']]
odbclusters.to_csv(f'{clusters_file}.odb', sep='\t', index=None)

## Enrich KEGG

In [ ]:
kegg_file = f'{transitions_file}.odb' # columns = description, id, db, og

In [ ]:
transitionsogs = transitionstoogs(transitions_file)

# filter kegg ids (e.g. metabolism)
filton = '00'
# filter out KEGG macro classes
filterkegg = ['Metabolic pathways']

kegg = pd.read_table(kegg_file)
kegg = kegg.drop_duplicates(['og','id'])
kegg = kegg[(kegg['db'].str.contains('KEGG'))&~(kegg['description'].isin(filterkegg))]
kegg = kegg[kegg['id'].apply(lambda x: re.split('(\d+)', x)[1]).str.startswith(filton)] if filton else kegg

transitionsogskegg = pd.merge(transitionsogs, kegg, on=['og','ogname'])
transitionsogskegg['log10pval'] = transitionsogskegg['pval'].apply(lambda x: -np.log10(x))
transitionsogskegg['log10adjpval'] = transitionsogskegg['adjpval'].apply(lambda x: -np.log10(x))
transitionsogskegg = transitionsogskegg.reset_index(drop=True).sort_values('log10pval', ascending=False)

# -log10 pval scaling
transitionsogskegg = scalepval(transitionsogskegg, 'log10pval', 2, None)

transitionsogskegg.to_csv(f'{kegg_file}.kegg', sep='\t', index=None)

transitionsogskegg['log10pvalscaled'].plot.hist(bins=100, figsize=(3.1,3.1))

### KEGG ranking file

In [ ]:
transitionsogskeggfilt = transitionsogskegg[['og','log10pvalscaled','id','description','ogname']]
transitionsogskeggfilt.columns = ['name','score','feature','description','extraname']

transitionsogskeggfilt.to_csv(f'{kegg_file}.kegg.ranking', sep='\t', index=None)

### KEGG Enrichment

In [ ]:
file = f'{kegg_file}.kegg.ranking'
!utilities/enrich.R --input_file {file} --pval_cutoff 1

### Figure 2f

In [ ]:
colorsfrommap = [plt.cm.jet(n)[0:3] for n in np.array(list(range(0,12)))/11]

viridis = plt.cm.get_cmap('viridis', 256)
newcolors = np.array([(1,0.7,0.4,i/255) for i in range(255)])
newcmp3 = matplotlib.colors.ListedColormap(newcolors)

In [ ]:
enrfile = f'{file}.enriched'
pval = 0.05
maxgr = 6
save = True

plotenrichment(enrfile, file, newcmp3,  f'NES KEGG Meta', pval, maxgr, save)

# Clusters analysis

## Clusters scores

In [ ]:
# columns = description, id, db, og
kegg_file = f'{clusters_file}.odb'

goslim_file = f'{ogunigotrans_file}.slim'.replace('transitions','ogs')

# filter kegg ids (e.g. metabolism)
filton = '00'
# filter out kegg macro classes
filterkegg = ['Metabolic pathways']
# filter out go macro classes
filtergo = ['biological process','molecular function','cellular component', 'protein binding']

In [ ]:
clusters = pd.read_table(clusters_file)

kegg = pd.read_table(kegg_file)
kegg = kegg.drop_duplicates(['og','id'])
kegg = kegg[(kegg['db'].str.contains('KEGG'))]
kegg = kegg[~(kegg['description'].isin(filterkegg))]

keggmeta = kegg[kegg['id'].apply(lambda x: re.split('(\d+)', x)[1]).str.startswith(filton)]

goslim = pd.read_table(goslim_file)
goslim = goslim[~goslim['goslimname'].isin(filtergo)]

In [ ]:
def groupfeatures(df, tag):
    ''' for each user defined feature, calculates for each cluster the percentage of most frequent ones '''

    dfs = []
    for c in set(df['cluster']):

        dfc = df[df['cluster']==c].drop_duplicates()

        dfes, dnas = [], []
        for f in set(dfc['feature']):
            dfe = dfc[dfc['feature']==f]
            dfes.append((dfe.iloc[0]['feature'], len(dfe)))
            dnas.append((dfe.iloc[0]['featurename'], len(dfe)))

        dfcfeat = pd.DataFrame(dfes, columns = ['feature', 'og'])
        dfcnames = pd.DataFrame(dnas, columns = ['featurename', 'og'])

        features = ' --- '.join(dfcnames.sort_values('og', ascending=False)['featurename'].values)
        frequencies = ' --- '.join(map(str, dfcnames.sort_values('og', ascending=False)['og'].values))
        annotatedogs = len(list(set(dfc['og'])))
        population = dfc['n'].iloc[0]
        frequency = dfcfeat.max()['og']
        mostfreqfeat = ' --- '.join(dfcfeat[dfcfeat['og']==frequency]['feature'].tolist())
        mostfreqname = ' --- '.join(dfcnames[dfcnames['og']==frequency]['featurename'].tolist())
        
        # at least 2 og annotated
        score = round(frequency/annotatedogs, 2) if not annotatedogs <= 1 else 0

        dfs.append([c, population, tag, annotatedogs, frequency, score, 
                    mostfreqname, mostfreqfeat, features, frequencies])

    columns = ['cluster','pop','db','pop.ann','n.mostfreq','score',
               'mostfreq','mostfreqfeat','allfeat','n.allfeat']
    
    return pd.DataFrame(dfs, columns = columns)

# standardize datasets header
go = goslim[['og','ogname','goslim','goslimname','goslimtype']]
go.columns = ['og','ogname','feature','featurename','type']

ognames = dict(clusters[['og','name']].values)

# all keggs available
kegg['ogname'] = kegg['og'].apply(lambda x: ognames.get(x))
kegg = kegg[['og','ogname','id','description']].assign(type='K')
kegg.columns = ['og','ogname','feature','featurename','type']

# only metabolism associated kegg 
keggmeta['ogname'] = keggmeta['og'].apply(lambda x: ognames.get(x))
keggmeta = keggmeta[['og','ogname','id','description']].assign(type='Km')
keggmeta.columns = ['og','ogname','feature','featurename','type']

dfK = groupfeatures(pd.merge(clusters, kegg, on='og'), 'K')
dfKm = groupfeatures(pd.merge(clusters, keggmeta, on='og'), 'Km')
dfP = groupfeatures(pd.merge(clusters, go[go['type']=='P'], on='og'), 'P')
dfF = groupfeatures(pd.merge(clusters, go[go['type']=='F'], on='og'), 'F')
dfC = groupfeatures(pd.merge(clusters, go[go['type']=='C'], on='og'), 'C')
# string = pd.read_table(f'{string_outfile}.networkx')

# final = pd.concat([dfK,dfKm,dfP,dfF,dfC,string]).sort_values(
final = pd.concat([dfK,dfKm,dfP,dfF,dfC]).sort_values(
    ['cluster','db'], ascending=[True,False])
final['score'] = [x[1] if x[0] >1 else 0
                  for x in final[['n.mostfreq','score']
                                ].values.tolist()]

clusters = clusters.dropna(subset=['cluster','n','og','name'])
cluog = dict(clusters.groupby('cluster')['og'].apply(list).apply(lambda x: ' --- '.join(x)).reset_index().values)
cluna = dict(clusters.groupby('cluster')['name'].apply(list).apply(lambda x: ' --- '.join(x)).reset_index().values)
final['ogs'] = final['cluster'].apply(lambda x: cluog.get(x))
final['ognames'] = final['cluster'].apply(lambda x: cluna.get(x))

final.to_csv(f'{clusters_file}.scores', sep='\t', index=None)

### Supplementary figure 7g

In [ ]:
final = pd.concat([dfK,dfKm,dfP,dfF,dfC]).sort_values(
    ['cluster','db'], ascending=[True,False])
final['annscore'] = final['pop.ann']/final['pop']
final['annscore'] = final['annscore'].apply(lambda x: round(x, 2))
final = final.reset_index(drop=True)

maxn = 5
final['pop'] = final['pop'].apply(lambda x: maxn if x >= maxn else x)

plt.rcParams['axes.linewidth'] = 0.8
plt.rcParams['lines.linewidth'] = 0.8
plt.rcParams['font.size'] = 8
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['legend.fontsize'] = 6

fig, axes = plt.subplots(
    1,len(set(final['pop'])),
    figsize=(len(set(final['pop']))*1, 3.1), 
    dpi=100, sharey=True)

for i, ax in zip(set(final['pop']), axes):
    for db, col in zip(['P','C','F','K','Km'], 
                       [(0,0.75,0.75),(0.75,0.75,0),
                        (0.75,0,0.75),(1,0.5,0.5),
                        (1,0.7,0.4)]):
        
        d = final[final['db']==db][final['pop']==i]
        d[db] = d['annscore']
        d.sort_values(db)[db].plot.kde(color=col, ax=ax)
    
    ax.spines[['top','right','left']].set_visible(False)
    ax.set_xticks([0, 0.5, 1])
    ax.set_xlim(0,1)
    ax.tick_params(axis='y', which='both', left=False)
    
    ax.set_xlabel(i)
    if i == maxn:
        ax.set_xlabel('≥'+str(i))

axes[0].spines[['left']].set_visible(True)
axes[-1].spines[['bottom']].set_visible(True)
plt.ylim(0,3.5)
plt.xlim(0,1)

plt.legend(['BP','CC','MF','KEGG','KEGG_Meta'], 
           loc=(-0.2, 0.7), frameon=False, title='Database')

fig.tight_layout()
fig.savefig(f'images/{clusters_file}_annotationscore_multi.pdf')

In [ ]:
final = pd.concat([dfK,dfKm,dfP,dfF,dfC]).sort_values(
    ['cluster','db'], ascending=[True,False])
final['annscore'] = final['pop.ann']/final['pop']
final['annscore'] = final['annscore'].apply(lambda x: round(x, 2))
final = final.reset_index(drop=True)
fig, axes = plt.subplots(1,1,figsize=(3.1,3.1), dpi=150, sharex=True)

for db, col in zip(['P','C','F','K','Km'], [(0,0.75,0.75),(0.75,0.75,0),(0.75,0,0.75),(1,0.5,0.5),(1,0.7,0.4)]):

    d = final[final['db']==db]
    d[db] = d['annscore']
    d.sort_values(db)[db].plot.kde(color=col, ax=axes)
axes.spines[['right','top']].set_visible(False)
plt.legend(['BP','CC','MF','KEGG','KEGG_Meta'], loc='upper left', frameon=False, title='Database')
fig.tight_layout()
plt.ylim(0,3)
plt.xlim(0,1)
fig.savefig(f'images/{clusters_file}_annotationscore_single.pdf')

## Supplementary figure 7h

Uncomment the cell below to generate `external/ko_og_pathway.tsv` (it takes about 2,5 hours)

In [ ]:
# !python3 utilities/ko2og.py 

In [ ]:
clusters_file_scores = f'{clusters_file}.scores'
clusters = pd.read_table(clusters_file_scores)

db = 'Km'

clusters = clusters[clusters['db']==db][clusters['n.mostfreq']>1]
clusters['mostfreqfeat'] = clusters['mostfreqfeat'].apply(lambda x: x.split(' --- '))

perc_thr = 50

clusters['perc.ann'] = clusters['pop.ann']*100/clusters['pop']
clusters = clusters[clusters['perc.ann']>=perc_thr]

koec = pd.read_table(f"https://rest.kegg.jp/link/enzyme/ko", header=None, names=['ko','ec'])
mapko = pd.read_table(f"https://rest.kegg.jp/link/ko/path", header=None, names=['map','ko'])

maps = pd.merge(koec, mapko, how='outer')
maps['ko'] = maps['ko'].str[3:]
maps['map'] = maps['map'].str[5:]

koog = pd.read_table(koog_file)

ecmaps = pd.merge(koog, maps)

clumap = clusters.explode('mostfreqfeat')[['cluster', 'pop', 'n.mostfreq', 'mostfreqfeat','ogs']]
clumap['ogs'] = clumap['ogs'].apply(lambda x: x.split(' --- '))
clumap = clumap.explode('ogs')
clumap = clumap.rename(columns={'mostfreqfeat': 'map', 'ogs': 'og'})

clumap = pd.merge(clumap, ecmaps, how='left')

counts = []
for c in set(clusters['cluster']):
    for m in set(clumap[clumap['cluster']==c]['map']):
        # aggiungere il dropna per gli EC non presenti?
        counts.append([c, clumap[clumap['cluster']==c]['pop'].iloc[0], m, len(ecmaps[ecmaps['map']==m].drop_duplicates('ec').dropna()),
         len(clumap[clumap['cluster']==c].drop_duplicates('ec').dropna())])
counts = pd.DataFrame(counts)
counts[5] = counts[4]/counts[3]
counts.columns = ['cluster','pop','map','lenmap','lenec','ECcov']
counts.replace([np.inf, -np.inf], np.nan, inplace=True)
counts = counts.dropna()

clusters = clusters.explode(['mostfreqfeat'])
clusters = clusters[['cluster','pop','n.mostfreq','mostfreqfeat']]
clusters = clusters.groupby('mostfreqfeat')['n.mostfreq'].sum().reset_index()
clusters.columns = ['map','nfreq']

In [ ]:
save = True

plt.rcParams['axes.linewidth'] = 0.8
plt.rcParams['lines.linewidth'] = 0.8
plt.rcParams['font.size'] = 8
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['legend.fontsize'] = 6

#garbage graph
fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
fig.write_image("random.pdf")


df = pd.merge(counts, clusters)
df = df.sort_values('ECcov', ascending=False)
keggnames = dict(keggmeta[['feature','featurename']].values)
df['mapname'] = df['map'].apply(lambda x: keggnames.get(x))
df = df.sort_values(['nfreq', 'ECcov'], ascending=[False,False])

def customwrap(s, width=15):
    return "<br>".join(textwrap.wrap(s,width=width))

df['mapname'] = df['mapname'].apply(customwrap)
df = df[df['nfreq']>=2] # default 4

df = pd.merge(df.groupby('mapname')['ECcov'].sum().reset_index(),
              df.groupby('mapname')['nfreq'].apply(list).apply(lambda x: x[0]).reset_index())

fig = px.treemap(df, path=[px.Constant("Clusters - KEGG Meta"), 'mapname'], values='nfreq',
                  color='ECcov', hover_data=['ECcov'],
                  color_continuous_scale='OrRd',
                 width=720, height=325)

percents = (100*df.ECcov / sum(df.ECcov)).tolist()
salaries = df.ECcov.tolist()
fig.data[0].customdata = np.column_stack([percents, salaries])
fig.data[0].texttemplate = "%{label}<br>nfreq: %{value}<br>ECcov: %{customdata[1]:.2f}"

fig.update_layout(margin = dict(t=5, l=5, r=5, b=5),
                  uniformtext=dict(minsize=6))
fig.show(renderer='notebook')

if save == True:
    fig.write_image(f'images/{clusters_file}.ECcoverage.pdf')
    
!rm random.pdf

## Supplementary figure 7j

In [ ]:
scores_file = f'{clusters_file}.scores'

In [ ]:
# plotting clusters with at least 50% of orthogroup annotated
clusters = pd.read_table(clusters_file)
scores = pd.read_table(scores_file)
perc_thr = 50

clusterslen = clusters.drop_duplicates('cluster').groupby('n')['cluster'].apply(len).reset_index().rename(columns={'n':'pop'})

dfs = [clusterslen]
# for d in ['P','C','F','K','Km','S']:
for d in ['P','C','F','K','Km']:
    scores['perc.ann'] = scores['pop.ann']*100/scores['pop']
    scoresann = scores[scores['db']==d][scores['perc.ann']>=perc_thr]
    clusterslendb = scoresann.groupby('pop')['cluster'].apply(len).reset_index()
    clusterslendb = clusterslendb.rename(columns={'cluster':d})
    dfs.append(clusterslendb)
    
dfsmerg = reduce(lambda left, right: 
                   pd.merge(left, right,on=['pop'], how='outer'), dfs)
dfsmerg = dfsmerg.rename(columns={'pop':'clustersize', 'cluster':'count'})

In [ ]:
uplim = 5 # default 10
save = True

plt.rcParams['axes.linewidth'] = 0.8
plt.rcParams['lines.linewidth'] = 0.8
plt.rcParams['font.size'] = 8
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['legend.fontsize'] = 6

dfsmerg1 = dfsmerg[dfsmerg['clustersize']<uplim]
dfsmerg1.loc[8] = [uplim]+dfsmerg[
    dfsmerg['clustersize']>=uplim].sum(axis=0).tolist()[1:]

fig, (ax2) = plt.subplots(1, 1, figsize=(3.1,3.1), 
                          dpi=100, sharex=True)

colorsfrommap = [plt.cm.jet(n)[0:3] for n in 
                 np.array(list(range(0,12)))/11]

for i in range(2, uplim+1):

    if i in set(dfsmerg1['clustersize']):
        n2 = dfsmerg1[dfsmerg1['clustersize']==i]

        ax2.bar(i, n2['count'].iloc[0],  0.8, label='Total', color='gainsboro')
        # ax2.bar(i-0.3, n2['S'].iloc[0],  0.15, label='String', color=(0.2,0.5,0.2,0.8))
        ax2.bar(i-0.3, n2['C'].iloc[0],  0.2, label='CC',        color=(0.75,0.75,0)) 
        ax2.bar(i-0.1, n2['P'].iloc[0],  0.2, label='BP',        color=(0, 0.75, 0.75))
        ax2.bar(i+0.1, n2['K'].iloc[0],  0.2, label='KEGG',      color=(0.75, 0, 0))
        ax2.bar(i+0.3, n2['Km'].iloc[0], 0.2, label='KEGG_Meta', color=(1, 0.65, 0))

        plt.xlim(1, uplim+1)
        ax2.set_xticks(list(range(2, uplim+1)))
        labels = list(map(str, list(range(2, uplim+1))))
        labels[-1] = f'>{uplim}'
        ax2.set_xticklabels(labels)
        if i == 2:
            ax2.legend(loc='upper right', frameon=False)
        
ax2.spines['top'].set_visible(False)
ax2.xaxis.tick_bottom()
ax2.spines[['top', 'right']].set_visible(False)
ax2.set_xlabel('Cluster size')
ax2.set_ylabel('# Clusters')

plt.subplots_adjust(hspace=0.1)

if save == True:
    fig.tight_layout()
    fig.savefig(f'images/{clusters_file}.distributions.pdf')

## Figure 3c

In [ ]:
# plotting the scores for each cluster

plt.rcParams['axes.linewidth'] = 0.8
plt.rcParams['lines.linewidth'] = 0.8
plt.rcParams['font.size'] = 8
plt.rcParams['xtick.labelsize'] = 6
plt.rcParams['ytick.labelsize'] = 6
plt.rcParams['legend.fontsize'] = 6

save = True

scores = pd.read_table(f'{clusters_file}.scores')
# DBC score
scores['pop.ann.perc'] = scores['pop.ann']/scores['pop']
scores = scores[scores['pop.ann.perc']>0.5] # at least 50% annotated og in each cluster
scores = scores[~scores['mostfreq'].astype(str).str.contains('protein binding', case=False)]

ann = {'K': 'K', 'Km': 'KM', 'S': 'S', 'F': 'MF', 'P': 'BP', 'C': 'CC'}

df2 = pd.concat([pd.DataFrame([[ann.get(d), len(scores[scores['db']==d][scores['score']==1].drop_duplicates('cluster')), 's=1'],
    [ann.get(d), len(scores[scores['db']==d][scores['score']==0].drop_duplicates('cluster')), 's=0'],
                              [ann.get(d), len(scores[scores['db']==d][(scores['score']>0)&(scores['score']<1)].drop_duplicates('cluster')), '0<s<1'],]) for d in ['K','Km','S','P','C','F']])
df2.columns = ['type','count','slice']
df2 = df2.reset_index(drop=True)

fig, ax = plt.subplots(figsize=(3.1,3.1), dpi=100)
for e, (colo, typ) in enumerate(zip([(0.75,0.75,0),(0,0.75,0.75),(0.75,0,0.75),(1,0.7,0.4),(1,0.7,0.4)], ['CC','BP','MF','K','KM'])):
    ax.bar(e-0.25, df2[df2['type']==typ][df2['slice']=='s=1']['count'], width=0.25, color=colo, alpha=0.2)
    ax.bar(e, df2[df2['type']==typ][df2['slice']=='s=0']['count'], width=0.25, color=colo, alpha=0.5)
    ax.bar(e+0.25, df2[df2['type']==typ][df2['slice']=='0<s<1']['count'], width=0.25, color=colo, alpha=1)
    
ax.bar(0,0,0,color='gainsboro',label='1')
ax.bar(0,0,0,color='gray',label='0')
ax.bar(0,0,0,color='black',label='(0,1)')

ax.set_xlabel('Database')
ax.set_ylabel('# Clusters')

ax.spines[['top','right']].set_visible(False)
ax.set_xticklabels(['-','CC','BP','MF','KEGG','KEGG_Meta'])
plt.legend(loc=(0.75,0.68), title='DBC score', frameon=False)

ax.annotate('Gene Ontology', xy=(0.31, 1), xytext=(0.31, 1.02), xycoords='axes fraction', 
            fontsize=6, ha='center', va='bottom',color='gray',
            arrowprops=dict(arrowstyle='-[, widthB=7.42, lengthB=0.5',color='gray', lw=1.0))

ax.annotate('KEGG PATHWAY', xy=(0.788, 1), xytext=(0.788, 1.02), xycoords='axes fraction', 
            fontsize=6, ha='center', va='bottom',color='gray',
            arrowprops=dict(arrowstyle='-[, widthB=4.78, lengthB=0.5',color='gray', lw=1.0))

fig.savefig(f'images/{clusters_file}.annotation.scores.pdf')